# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pickle
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report, hamming_loss
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM test2', con=engine)
df.related.replace(2,1,inplace=True)
X = df.message.values 
Y = df.iloc[:,4:]
Y.tail()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
26215,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
       
        # Remove stop words
        if tok in stopwords.words("english"):
            continue
        # Reduce words to their stems
        tok = PorterStemmer().stem(tok)
        # Reduce words to their root form
        tok = lemmatizer.lemmatize(tok).lower().strip()

        clean_tokens.append(tok)
        
    clean_tokens = [tok for tok in clean_tokens if tok.isalpha()]
    return clean_tokens

print(X[3])
print(tokenize(X[3]))

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogan', 'destroy', 'onli', 'hospit', 'croix', 'function', 'need', 'suppli', 'desper']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
 # train classifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

In [7]:
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)

related
             precision    recall  f1-score   support

          0       0.64      0.46      0.54      1575
          1       0.84      0.92      0.88      4979

avg / total       0.79      0.81      0.80      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5460
          1       0.79      0.42      0.55      1094

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3889
          1       0.75      0.60      0.67      2665

avg / total       0.75      0.76      0.75      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      0.99      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [9]:
parameters = {
        'vect__max_df':[0.75,1.0],
        'clf__estimator__n_estimators': [20, 50]
    }


cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10, n_jobs=4)
cv.fit(X_train, Y_train) 

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=20, vect__max_df=0.75 ..............
[CV] clf__estimator__n_estimators=20, vect__max_df=0.75 ..............
[CV] clf__estimator__n_estimators=20, vect__max_df=0.75 ..............
[CV] clf__estimator__n_estimators=20, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=1.0, score=0.25663716814159293, total=13.5min
[CV] clf__estimator__n_estimators=20, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.75, score=0.25267012511443393, total=13.5min
[CV] clf__estimator__n_estimators=20, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.75, score=0.25663716814159293, total=13.5min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75 ..............
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.75, score=0.24992371071101618, total=13.6min
[CV] clf__estimator__n_estimators=50, vect__max_

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 39.6min


[CV]  clf__estimator__n_estimators=20, vect__max_df=1.0, score=0.2488556606652426, total=13.6min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, score=0.2638083613060726, total=20.6min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............


[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 47.3min remaining: 33.8min


[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, score=0.25358559658223984, total=20.7min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0 ...............
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, score=0.26731766859932865, total=20.4min


[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 67.0min remaining: 22.3min


[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.25770521818736647, total=20.4min
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.2540433323161428, total=20.3min
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, score=0.2697589258468111, total=19.9min


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 71.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'vect__max_df': [0.75, 1.0], 'clf__estimator__n_estimators': [20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
Y_pred = cv.predict(X_test)
for ix, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,ix]))

avg = (Y_pred == Y_test).mean().mean()
print("Accuracy Overall:\n", avg)

related
             precision    recall  f1-score   support

          0       0.71      0.41      0.52      1575
          1       0.83      0.95      0.89      4979

avg / total       0.80      0.82      0.80      6554

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5460
          1       0.85      0.48      0.61      1094

avg / total       0.89      0.90      0.89      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3889
          1       0.76      0.67      0.71      2665

avg / total       0.78      0.78      0.78      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Improve  the pipeline
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:

pipeline2.get_params()

In [ ]:
#Train & predict
pipeline2.fit(X_train, y_train)
perf_report(pipeline2, X_test, y_test)

In [ ]:
#Param tunning 
parameters2 = { #'vect__ngram_range': ((1, 1), (1, 2)), 
              #'vect__max_df': (0.5, 1.0), 
              #'vect__max_features': (None, 5000), 
              'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__learning_rate': [1,2] }

In [ ]:
cv2 = GridSearchCV(pipeline2, param_grid=parameters2)
cv2

### 9. Export your model as a pickle file

In [11]:
pickle.dump(cv, open("classifier.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.